# Cosyne 2019 NWB:N Tutorial - Extracellular Electrophysiology

## Set up NWB file
NWB files require a session start time to be entered with a timezone field.

In [1]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz('US/Pacific'))

nwbfile = NWBFile(identifier='Mouse5_Day3',
                 session_description='mouse in open exploration and theta maze',  # required
                 session_start_time=start_time,  # required
                 experimenter='My Name',  # optional
                 session_id='session_id',  # optional
                 institution='University of My Institution',  # optional
                 lab='My Lab Name',  # optional
                 related_publications='DOI:10.1016/j.neuron.2016.12.011')  # optional

## Subject info

In [2]:
from pynwb.file import Subject

nwbfile.subject = Subject(age='9 months', description='mouse 5',
                          species='Mus musculus', sex='M')

## Position
The `Position` object is a `MultiContainerInterface` that holds one or more `SpatialSeries` objects, which are a subclass of `TimeSeries`. Here, we put a `SpatialSeries` object called `'position'` in a `Position` object, and put that in a `ProcessingModule` named `'behavior'`.
<img src="images/position.png" width="800">

In [3]:
import numpy as np
from pynwb.behavior import SpatialSeries, Position

position_data = np.array([
    np.linspace(0, 10, 100),
    np.linspace(1, 8, 100)]).T
spatial_series_object = SpatialSeries(
   name='position', data=position_data,
   reference_frame='unknown',
   conversion=1.0, resolution=np.nan,
   timestamps=np.linspace(0, 10, 100))

In [4]:
pos_obj = Position(spatial_series=spatial_series_object)
behavior_module = nwbfile.create_processing_module(
    name='behavior',
    description='data relevant to behavior')

behavior_module.add_data_interface(pos_obj)

## Write to file

In [5]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('test_ephys.nwb', 'w') as io:
    io.write(nwbfile)

## Trials
Trials is another `DynamicTable` that lives an `/intervals/trials`.

In [6]:
nwbfile.add_trial_column('correct', description='correct trial')
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

## Electrodes table
Extracellular electrodes are stored in a `electrodes`, which is a `DynamicTable`. `electrodes` has several required fields: x, y, z, impedence, location, filtering, and electrode_group. Here, we also demonstate how to add optional columns to a table by adding the `'label'` column.<img src="images/electrodes_table.png" width="300">

In [7]:
nwbfile.add_electrode_column('label', 'label of electrode')
shank_channels = [4, 3]

electrode_counter = 0
device = nwbfile.create_device('implant')
for shankn, nelecs in enumerate(shank_channels):
    electrode_group = nwbfile.create_electrode_group(
       name='shank{}'.format(shankn),
       description='electrode group for shank {}'.format(shankn),
       device=device,
       location='brain area')
    for ielec in range(nelecs):
        nwbfile.add_electrode(
           x=5.3, y=1.5, z=8.5, imp=np.nan,
           location='unknown', filtering='unknown',
           group=electrode_group,
           label='shank{}elec{}'.format(shankn, ielec))
        electrode_counter += 1

all_table_region = nwbfile.create_electrode_table_region(
  list(range(electrode_counter)), 'all electrodes')

## LFP
`LFP` is another `MultiContainerInterface`. It holds one or more `ElectricalSeries` objects, which are `TimeSeries`. Here, we put an `ElectricalSeries` named `'lfp'` in an `LFP` object, in a `ProcessingModule` named `'ecephys'`.
<img src="images/lfp.png" width="800">

In [8]:
from pynwb.ecephys import ElectricalSeries, LFP
lfp_data = np.random.randn(100, 7)
ecephys_module = nwbfile.create_processing_module(
    name='ecephys',
    description='extracellular electrophysiology data')
ecephys_module.add_data_interface(
LFP(ElectricalSeries('lfp', lfp_data, all_table_region, 
rate=1000., resolution=.001, conversion=1.)))

## Spike Times
Spike times are stored in another `DynamicTable` of subtype `Units`. The main `Units` table is at `/units` in the HDF5 file. You can add columns to the `Units` table just like you did for `electrodes`.

In [9]:
for shankn, channels in enumerate(shank_channels):
    for n_units_per_shank in range(np.random.poisson(lam=5)):
        n_spikes = np.random.poisson(lam=10)
        spike_times = np.abs(np.random.randn(n_spikes))
        nwbfile.add_unit(spike_times=spike_times)

## Write and read
Data arrays are read passively from the file. That means `TimeSeries.data` does not read the entire data object, but presents an h5py object that can be indexed to read data. Index this array just like a numpy array to read only a specific section of the array, or use the `[:]` operator to read the entire thing.

In [10]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('test_ephys.nwb', 'w') as io:
    io.write(nwbfile)

with NWBHDF5IO('test_ephys.nwb', 'r') as io:
    nwbfile2 = io.read()

    print(nwbfile2.modules['ecephys']['LFP'].electrical_series['lfp'].data[:])

[[ 1.03563717e-01 -2.76371741e-01 -2.19908957e+00  5.93910486e-02
   7.29876814e-01  1.89061388e+00  1.08410954e+00]
 [ 3.05150577e-01 -1.79707528e+00  7.94343700e-02 -1.68948470e+00
   1.08286493e+00  4.68934171e-01  9.65363098e-01]
 [-2.50233526e-01  3.17538057e-01  4.20061557e-01  1.27907468e+00
   1.26365882e-01  5.36872560e-01 -2.10248951e+00]
 [-1.07600402e+00  5.29935396e-01 -9.47984273e-01 -6.34994076e-01
  -1.07486186e-01  4.01802151e-02 -1.31972766e+00]
 [-3.77857124e-01  6.49282171e-01 -5.36489615e-02  7.29251627e-01
  -1.61174750e+00  1.27612603e+00  1.65178982e+00]
 [-1.27633167e-01  5.10753166e-01  1.90419288e-01  8.13830932e-01
  -5.26659481e-01  9.81218361e-01  5.95998476e-01]
 [ 1.73287974e-01 -2.84082143e+00  9.74667674e-02 -7.92348073e-02
  -1.74263922e+00 -5.08060619e-01 -3.85113429e-01]
 [ 1.28224693e+00 -2.54251984e-01 -5.43770248e-01 -7.95895733e-01
  -1.44569451e+00  2.58467208e-02 -3.28959478e-01]
 [-2.58004394e+00 -5.16284642e-01  1.63926238e-01  6.61782642e-0

## Accessing data regions
You can easily read subsections of datasets

In [11]:
io = NWBHDF5IO('test_ephys.nwb', 'r')
nwbfile2 = io.read()

print('section of lfp:')
print(nwbfile2.modules['ecephys']['LFP'].electrical_series['lfp'].data[:10,:5])
print('')
print('')
print('spike times from first unit:')
print(nwbfile2.units['spike_times'][0])
io.close()

section of lfp:
[[ 0.10356372 -0.27637174 -2.19908957  0.05939105  0.72987681]
 [ 0.30515058 -1.79707528  0.07943437 -1.6894847   1.08286493]
 [-0.25023353  0.31753806  0.42006156  1.27907468  0.12636588]
 [-1.07600402  0.5299354  -0.94798427 -0.63499408 -0.10748619]
 [-0.37785712  0.64928217 -0.05364896  0.72925163 -1.6117475 ]
 [-0.12763317  0.51075317  0.19041929  0.81383093 -0.52665948]
 [ 0.17328797 -2.84082143  0.09746677 -0.07923481 -1.74263922]
 [ 1.28224693 -0.25425198 -0.54377025 -0.79589573 -1.44569451]
 [-2.58004394 -0.51628464  0.16392624  0.66178264 -0.90562619]
 [ 1.33867375  0.31490481  0.39328047  1.70600055 -0.16006067]]


spike times from first unit:
[0.09028211 0.90680742 1.8428306  0.23849214 0.39276416 0.91001347
 0.53436834 0.26680888 1.02365249]


In [12]:
from nwbwidgets import nwb2widget

In [13]:
nwb2widget(nwbfile)